In [100]:
# For LZW coding, we need a sequence of numbers, 
# Keep all grayscale images in images folder
# LZW contains the text files generated
# LZW_graphs contains all the graphs generated
# LZW_results contains the results of the before and after compression

In [101]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
import os
from scipy.stats import entropy


Analysis Functions

In [102]:
def fixImageDimensions(img, blockSize):
    rows, cols = img.shape[0], img.shape[1]

    if(rows%blockSize != 0):
        newRows = ((rows//blockSize)+1)*blockSize
        img = np.pad(img, ((0, newRows-rows), (0, 0)), 'constant', constant_values=(0, 0))

    if(cols%blockSize != 0):
        newCols = ((cols//blockSize)+1)*blockSize
        img = np.pad(img, ((0,0), (0, newCols-cols)), 'constant', constant_values=(0, 0))
    return img

In [103]:
def getEntropy(img):
    # first get the histogram of the image
    N = 256
    hist, _ = np.histogram(img, N, range=(0,N-1))
    p_dist = hist/np.sum(hist)
    en = entropy(p_dist, base=2)
    return en
    

In [104]:
def getCorrectCodeDict(codeDict):
    newDict = {}
    for key in codeDict:
        if isinstance(key, int):
            newDict[key] = codeDict[key]
    return newDict

In [105]:
def CalculateCompressionRatio(img, encodedSequence, blockSize):
    # bits in originalImage/bits in encodedSequence
    rows, cols = img.shape[0], img.shape[1]    
    # calculating bits in original image
    originalBits = rows*cols
    compressedBits = 0
    
    if(blockSize==-1):
        compressedBits = len(encodedSequence)
        return originalBits/compressedBits
    
    for i in encodedSequence:
        compressedBits += len(i)
    
    return originalBits/compressedBits


In [106]:
# calculating RMSE
def CalculateRMSE(originalImage, reconstructedImage):
    if(len(originalImage.shape)==3):
        # Convert the image to RGB for saving
        originalImage = cv.cvtColor(originalImage, cv.COLOR_YCR_CB2BGR)
    
    if(len(reconstructedImage.shape)==3):
        # Convert the image to RGB for saving
        reconstructedImage = reconstructedImage.astype('uint8')
        reconstructedImage = cv.cvtColor(reconstructedImage, cv.COLOR_YCR_CB2BGR)
        
    rmse = math.sqrt(np.mean((originalImage - reconstructedImage) ** 2))
    return rmse

In [171]:
def lzw_encode(img, codeSize):
    rows, cols = img.shape[0],img.shape[1]
    # initialize the dictionary
    codeDict = {}
    for i in range(256):
        codeDict[i] = str(i)
        codeDict[str(i)] = i

    # initialize other variables needed for the table
    encodedSequence = np.array([], dtype=np.uint32)
    recognizedSequence = ""
    currentlyProcessingPixel = ""

    index = 256
    for i in range(0, rows):
        for j in range(0, cols):
            # at the beginning, we don't have any recognized sequence, so we simply get the currently processing pixel and set that to the current pixel we are at
            if recognizedSequence=="":
                # set currentlyProcessingPixel to the current pixel we are at
                currentlyProcessingPixel = str(img[i][j])
                # add this currentlyProcessingPixel to the recognizedSequence which was earlier empty
                recognizedSequence = currentlyProcessingPixel
            else:
                # set currentlyProcessingPixel to the current pixel we are at
                currentlyProcessingPixel = str(img[i][j])
                # if the sequence exists in our dict, great, we can simply work further
                if f"{recognizedSequence}_{currentlyProcessingPixel}" in codeDict:
                    # update the recognized sequence 
                    recognizedSequence = recognizedSequence + '_' + currentlyProcessingPixel
                else: # sequence does not exist in codeDict
                    # add the recognized sequence to the dictionary and it's index should be the current length of the dict, because indexes are stored from 0 to length-1, so next index is --> length
                    # if(index>=2**codeSize):
                    #     encodedSequence = np.append(encodedSequence, codeDict[f"{recognizedSequence}"])
                    #     # encodedSequence = np.append(encodedSequence, codeDict[f"{currentlyProcessingPixel}"])
                    #     recognizedSequence = currentlyProcessingPixel
                    #     # print("YES")
                    #     pass

                    newSequence = f"{recognizedSequence}_{currentlyProcessingPixel}"
                    codeDict[newSequence] = index
                    codeDict[index] = newSequence 
                    index+=1
                    # add the code of the recognized sequence to the encoded sequence
                    codeDictKey = str(recognizedSequence)
                    encodedSequence = np.append(encodedSequence, codeDict[codeDictKey])
                    
                    # reset the recognized sequence
                    recognizedSequence = currentlyProcessingPixel
            if(i+1==rows and j+1==cols):
                codeDictKey = str(recognizedSequence)
                encodedSequence = np.append(encodedSequence, codeDict[codeDictKey])
                break
    # codeDict = getCorrectCodeDict(codeDict)
    newCodeDict = {}
    for i in encodedSequence:
        newCodeDict[i] = codeDict[i]

    # print("Done Encoding!")
    return encodedSequence

            


In [172]:
def EncodeEntireImage(img, blockSize, codeSize, imagefileName):
    
    rows, cols = img.shape[0], img.shape[1]
    if(blockSize==-1):
        encodedSequenceArray = lzw_encode(img, codeSize)
        MaxCode = max(encodedSequenceArray)
        AvgLengthOfEncodedPixels = sum(encodedSequenceArray)/len(encodedSequenceArray)
        CompressionRatio = CalculateCompressionRatio(img, encodedSequenceArray,blockSize) 
        Entropy = getEntropy(img)
        return encodedSequenceArray, MaxCode, CompressionRatio, AvgLengthOfEncodedPixels, Entropy
    
    # initialize encodedSequence array for each block
    encodedSequenceArray =[]

    AvgLengthOfEncodedPixels = 0

    # get Entropy
    Entropy = getEntropy(img)

    # now break the image into nxn blocks 
    for i in range(0, rows, blockSize):
        for j in range(0, cols, blockSize):
            # get the encoded sequence and codedDictionary for a block
            encodedSequenceOfBlock = lzw_encode(img[i:i+blockSize, j:j+blockSize], codeSize)
            encodedSequenceArray.append(encodedSequenceOfBlock)

    # Find the max value of the codes, which will give you the maximum number of codes used
    MaxCode = max(max(LIST) for LIST in encodedSequenceArray)
    
    NumberOfCodes = 0

    # calculating avg length
    for i in encodedSequenceArray:
            AvgLengthOfEncodedPixels += len(i)
            NumberOfCodes += len(i)
            
    AvgLengthOfEncodedPixels /= len(encodedSequenceArray)
    
    CompressionRatio = CalculateCompressionRatio(img, encodedSequenceArray, blockSize)

    # convert the encodedSequence into a numpy array
    encodedSequenceArray = np.array(encodedSequenceArray, dtype=object)
    
    # save the encoded values to a csv file
    saveDestination = 'LZW/encoded_'+imagefileName+'.txt'
    np.savetxt(saveDestination,encodedSequenceArray, delimiter=',', fmt='%s')
    

    return encodedSequenceArray,saveDestination, MaxCode,NumberOfCodes, CompressionRatio, AvgLengthOfEncodedPixels, Entropy

Decoding


In [185]:
def lzw_decode(encodedSequence, blockSize):

    decodedSequence = []
    
    imgBlock = np.zeros((blockSize, blockSize), dtype='int32')

    # initialize the dictionary
    codeDict = {}

    # fill the dictionary with the first 256 values
    for i in range(256):
        codeDict[i] = str(i)
        codeDict[str(i)] = i

    # get the first pixel from the encoded sequence and them remove it from the encoded sequence, so that we can go to the next pixel
    currentPixel = str(encodedSequence[0])
    # add the first pixel to the decoded sequence
    decodedSequence.append(currentPixel)
    # remove the first pixel from the encoded sequence
    encodedSequence = encodedSequence[1:]
    # initialize the NewEntryIndex
    NewEntryIndex = 256

    # loop through the encoded sequence
    for item in encodedSequence:
        newEntry = ""

        # if the item is in the dictionary, then add it to the decoded sequence
        if item in codeDict:
            newEntry = codeDict[item]
        elif(item == NewEntryIndex): # if the item is equal to the NewEntryIndex, then we have reached at an edge, so we need to add a new sequence in our dictionary
            newEntry = currentPixel + '_' + currentPixel.split('_')[0]
        else:# else there is an error
            raise ValueError("This is not a valid encoded array, item does not exist in dictionary, nor is it equal to NewEntryIndex")

        # add the new entry to the decoded sequence
        decodedSequence.append(newEntry)

        
        stringToAdd = currentPixel + '_' + newEntry.split('_')[0]
        # add the new entry to the dictionary
        codeDict[NewEntryIndex] = stringToAdd
        codeDict[stringToAdd] = NewEntryIndex
        # increment for the next entry
        NewEntryIndex+=1
        # update pixel value
        currentPixel = newEntry

    decodedSequence = [int(j) for i in decodedSequence for j in i.split('_')]
    
    print(len(decodedSequence))
    # now we need to convert the decoded sequence into an image
    CurrIndex = 0
    for i in range(blockSize):
        for j in range(blockSize):
            imgBlock[i,j] = decodedSequence[CurrIndex]
            CurrIndex +=1
    
    return imgBlock
            

In [186]:
def DecodeEntireImage(encodedSequence, rows, cols, blockSize):
    
    img = np.zeros((rows, cols), dtype='int32')
    index = 0

    for i in range(0,rows, blockSize):
        for j in range(0, cols, blockSize):
            img[i:i+blockSize, j:j+blockSize] = lzw_decode(encodedSequence[index], blockSize)
            index += 1
            
    return img


In [ ]:
foldername = 'images/'
filename = 'lena.tif'
img = cv.imread(foldername+filename, 0)
blockSize = 8

img = fixImageDimensions(img, blockSize)
codeSize = 9
encodedSequenceArray,encodedFileDestination, MaxCode, NumberOfCodes, CompressionRatio, AvgLengthOfEncodedPixels, Entropy = EncodeEntireImage(img, blockSize, codeSize, imagefileName=filename)
print("MaxCode: ", MaxCode)
print("CompressionRatio: ", CompressionRatio)
print("AvgLengthOfEncodedPixels: ", AvgLengthOfEncodedPixels)
print("Entropy: ", Entropy)
print("NumberOfCodes: ", NumberOfCodes)
print('EncodedFileDestination: ', encodedFileDestination)

decodedImage = DecodeEntireImage(encodedSequenceArray, img.shape[0], img.shape[1], blockSize)
plt.imshow(decodedImage, cmap='gray')